# 03. ステレオ音声の可視化

ステレオ音声の左右チャンネルの違いや、空間的な音響効果を可視化します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. ステレオパンニング効果の可視化

In [ ]:
def create_panned_audio(signal, pan_position):
    """
    パンニング効果を適用
    pan_position: -1.0 (完全に左) から 1.0 (完全に右)
    """
    # パンニング係数の計算
    left_gain = np.sqrt((1 - pan_position) / 2)
    right_gain = np.sqrt((1 + pan_position) / 2)
    
    left_channel = signal * left_gain
    right_channel = signal * right_gain
    
    return left_channel, right_channel

# 基本信号の生成
sample_rate = 44100
duration = 2.0
frequency = 440

t = np.linspace(0, duration, int(sample_rate * duration), False)
base_signal = np.sin(2 * np.pi * frequency * t)

# 異なるパンニング位置
pan_positions = [-1.0, -0.5, 0.0, 0.5, 1.0]
pan_labels = ['完全に左', '左寄り', '中央', '右寄り', '完全に右']

plt.figure(figsize=(15, 12))

for i, (pan_pos, label) in enumerate(zip(pan_positions, pan_labels)):
    left, right = create_panned_audio(base_signal, pan_pos)
    
    plt.subplot(len(pan_positions), 1, i+1)
    
    # 波形の一部を表示
    samples_to_show = 2000
    plt.plot(t[:samples_to_show], left[:samples_to_show], 'b-', 
             label=f'左 (ゲイン: {np.sqrt((1-pan_pos)/2):.2f})', alpha=0.8)
    plt.plot(t[:samples_to_show], right[:samples_to_show], 'r-', 
             label=f'右 (ゲイン: {np.sqrt((1+pan_pos)/2):.2f})', alpha=0.8)
    
    plt.title(f'パンニング: {label}')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    
    if i == len(pan_positions) - 1:
        plt.xlabel('時間 (秒)')

plt.tight_layout()
plt.show()

## 2. ステレオ位相差による空間効果

In [ ]:
def create_phase_shifted_stereo(signal, phase_shift_samples):
    """
    位相差によるステレオ効果
    phase_shift_samples: 右チャンネルの遅延サンプル数
    """
    left_channel = signal.copy()
    right_channel = np.zeros_like(signal)
    
    if phase_shift_samples > 0:
        right_channel[phase_shift_samples:] = signal[:-phase_shift_samples]
    elif phase_shift_samples < 0:
        right_channel[:phase_shift_samples] = signal[-phase_shift_samples:]
    else:
        right_channel = signal.copy()
    
    return left_channel, right_channel

# 異なる位相差での効果
phase_shifts_ms = [0, 0.5, 1.0, 2.0, 5.0]  # ミリ秒
base_signal = np.sin(2 * np.pi * 1000 * t)  # 1kHz信号

plt.figure(figsize=(15, 12))

for i, phase_ms in enumerate(phase_shifts_ms):
    phase_samples = int(phase_ms * sample_rate / 1000)
    left, right = create_phase_shifted_stereo(base_signal, phase_samples)
    
    plt.subplot(len(phase_shifts_ms), 1, i+1)
    
    # 拡大表示用
    start_sample = 44100  # 1秒後から表示
    end_sample = start_sample + 200  # 短い区間
    
    plt.plot(t[start_sample:end_sample], left[start_sample:end_sample], 
             'b-', label='左チャンネル', linewidth=2)
    plt.plot(t[start_sample:end_sample], right[start_sample:end_sample], 
             'r-', label='右チャンネル', linewidth=2, alpha=0.8)
    
    plt.title(f'位相差: {phase_ms} ms ({phase_samples} サンプル)')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    
    if i == len(phase_shifts_ms) - 1:
        plt.xlabel('時間 (秒)')

plt.tight_layout()
plt.show()

## 3. ステレオ幅の可視化

In [ ]:
def calculate_stereo_width(left, right, window_size=1024):
    """
    ステレオ幅を計算
    """
    stereo_width = []
    
    for i in range(0, len(left) - window_size, window_size // 2):
        l_window = left[i:i+window_size]
        r_window = right[i:i+window_size]
        
        # 相関係数を計算
        correlation = np.corrcoef(l_window, r_window)[0, 1]
        
        # ステレオ幅 = 1 - 相関係数
        width = 1 - abs(correlation) if not np.isnan(correlation) else 0
        stereo_width.append(width)
    
    return np.array(stereo_width)

# 異なるステレオ効果の音声を生成
duration = 4.0
t = np.linspace(0, duration, int(sample_rate * duration), False)

# セクション1: モノラル（同じ信号）
mono_signal = np.sin(2 * np.pi * 440 * t)
left1 = mono_signal[:len(mono_signal)//4]
right1 = mono_signal[:len(mono_signal)//4]

# セクション2: 軽いステレオ（少し異なる周波数）
left2 = np.sin(2 * np.pi * 440 * t[len(t)//4:len(t)//2])
right2 = np.sin(2 * np.pi * 445 * t[len(t)//4:len(t)//2])

# セクション3: 強いステレオ（大きく異なる周波数）
left3 = np.sin(2 * np.pi * 440 * t[len(t)//2:3*len(t)//4])
right3 = np.sin(2 * np.pi * 880 * t[len(t)//2:3*len(t)//4])

# セクション4: 完全に独立した信号
np.random.seed(42)
left4 = np.random.normal(0, 0.5, len(t)//4)
right4 = np.random.normal(0, 0.5, len(t)//4)

# 全体の信号を結合
left_total = np.concatenate([left1, left2, left3, left4])
right_total = np.concatenate([right1, right2, right3, right4])

# ステレオ幅の計算
stereo_width = calculate_stereo_width(left_total, right_total)
time_width = np.linspace(0, duration, len(stereo_width))

plt.figure(figsize=(15, 12))

# 左チャンネル
plt.subplot(4, 1, 1)
plt.plot(t, left_total, 'b-', alpha=0.7)
plt.title('左チャンネル')
plt.ylabel('振幅')
plt.grid(True)
plt.axvline(x=1, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=2, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=3, color='r', linestyle='--', alpha=0.5)

# 右チャンネル
plt.subplot(4, 1, 2)
plt.plot(t, right_total, 'r-', alpha=0.7)
plt.title('右チャンネル')
plt.ylabel('振幅')
plt.grid(True)
plt.axvline(x=1, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=2, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=3, color='r', linestyle='--', alpha=0.5)

# ステレオ比較
plt.subplot(4, 1, 3)
plt.plot(t[:8000], left_total[:8000], 'b-', alpha=0.6, label='左')
plt.plot(t[:8000], right_total[:8000], 'r-', alpha=0.6, label='右')
plt.title('ステレオチャンネル比較（拡大表示）')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

# ステレオ幅
plt.subplot(4, 1, 4)
plt.plot(time_width, stereo_width, 'g-', linewidth=2)
plt.title('ステレオ幅の時間変化')
plt.xlabel('時間 (秒)')
plt.ylabel('ステレオ幅')
plt.grid(True)
plt.axvline(x=1, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=2, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=3, color='r', linestyle='--', alpha=0.5)

# 各セクションのラベル
plt.text(0.5, max(stereo_width)*0.8, 'モノラル', ha='center', fontsize=10, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
plt.text(1.5, max(stereo_width)*0.8, '軽いステレオ', ha='center', fontsize=10,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
plt.text(2.5, max(stereo_width)*0.8, '強いステレオ', ha='center', fontsize=10,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow"))
plt.text(3.5, max(stereo_width)*0.8, '独立信号', ha='center', fontsize=10,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral"))

plt.tight_layout()
plt.show()

## 4. ステレオイメージの円形表示

In [ ]:
def plot_stereo_circle(left_levels, right_levels, title="ステレオイメージ"):
    """
    ステレオ音声のバランスを円形で表示
    """
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # 円の描画
    circle = Circle((0, 0), 1, fill=False, color='black', linewidth=2)
    ax.add_patch(circle)
    
    # 中心線
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax.axvline(x=0, color='gray', linestyle='-', alpha=0.5)
    
    # ステレオポジションの計算と表示
    for i, (left, right) in enumerate(zip(left_levels, right_levels)):
        # 音量の正規化
        total_level = left + right
        if total_level > 0:
            # パンポジション (-1: 完全に左, 1: 完全に右)
            pan = (right - left) / total_level
            # 音量レベル（円の中心からの距離）
            radius = min(total_level, 1.0)
            
            # 座標計算
            x = pan * radius
            y = 0
            
            # プロット
            color = plt.cm.viridis(i / len(left_levels))
            ax.scatter(x, y, c=[color], s=100, alpha=0.7, 
                      label=f'時点{i+1}')
    
    # ラベル
    ax.text(-1.2, 0, 'L', fontsize=16, ha='center', va='center', 
            bbox=dict(boxstyle="circle", facecolor="lightblue"))
    ax.text(1.2, 0, 'R', fontsize=16, ha='center', va='center',
            bbox=dict(boxstyle="circle", facecolor="lightcoral"))
    
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=14)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()

# サンプルデータでステレオイメージを表示
sample_times = ['0-0.5s', '0.5-1s', '1-1.5s', '1.5-2s', '2-2.5s']
left_levels = [0.8, 0.6, 0.3, 0.1, 0.7]   # 左チャンネルレベル
right_levels = [0.2, 0.4, 0.7, 0.9, 0.3]  # 右チャンネルレベル

plot_stereo_circle(left_levels, right_levels, "ステレオバランスの変化")

# 各時点でのパンポジションを数値で表示
print("時間区間ごとのステレオバランス:")
print("時間\t\t左レベル\t右レベル\tパンポジション")
for i, (time, left, right) in enumerate(zip(sample_times, left_levels, right_levels)):
    total = left + right
    pan = (right - left) / total if total > 0 else 0
    print(f"{time}\t\t{left:.1f}\t\t{right:.1f}\t\t{pan:+.2f}")

## 練習問題

1. 時間とともにパンニングが左右に動く「オートパン」効果を作成してみましょう
2. ステレオ音声から左右の差分（Mid/Side処理）を計算して可視化してみましょう
3. 複数の楽器が異なる位置にパンニングされた混合音声を作成し、ステレオイメージを分析してみましょう